<a href="https://colab.research.google.com/github/Sinchana258/E-Waste-Management-System-using-deep-learning/blob/main/e_waste_classify.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade ultralytics




In [ ]:

!pip install  roboflow -q

from ultralytics import YOLO
import torch

print("✅ Torch version:", torch.__version__)
print("✅ CUDA available:", torch.cuda.is_available())

In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="DzUpdu4A39ELV7kMcSZy")
project = rf.workspace("sinchana-jqbdj").project("e-waste-classification-pm8bn")
version = project.version(6)
dataset = version.download("yolov11")


In [ ]:
from ultralytics import YOLO

try:
    model = YOLO("yolo11s.pt")
    print("Loaded YOLOv11 successfully!")
except Exception as e:
    print("❌ Error:", e)


In [ ]:
!ls /content/E-Waste-Classification-6


In [ ]:
!yolo train model=yolo11s.pt \
    data=/content/E-Waste-Classification-6/data.yaml \
    epochs=100 \
    imgsz=640 \
    batch=16 \
    amp=True \
    optimizer=auto \
    lr0=0.01 \
    cos_lr=True \
    name=ewaste_yolo11_v1


In [ ]:
!ls /content/runs/detect/ewaste_yolo11_v1

In [ ]:
!ls /content/runs/detect/ewaste_yolo11_v1/weights

In [ ]:
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt
import os

# ✅ Load your updated trained model (YOLOv11 run)
model = YOLO("/content/runs/detect/ewaste_yolo11_v1/weights/best.pt")

# Paths
test_path = "/content/E-Waste-Classification-6/test/images"
output_path = "/content/runs/detect/predict_annotated"
os.makedirs(output_path, exist_ok=True)

annotated_images = []

# Annotate images
for img_file in os.listdir(test_path):
    if img_file.lower().endswith((".jpg", ".jpeg", ".png")):
        img_path = os.path.join(test_path, img_file)

        # Run inference
        results = model.predict(source=img_path, save=False)

        # Load original image
        img = cv2.imread(img_path)
        if img is None:
            print(f"Failed to read image: {img_path}")
            continue

        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert to RGB for matplotlib

        for result in results:
            boxes = result.boxes
            for box in boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                conf = float(box.conf[0].item())
                cls_id = int(box.cls[0].item())
                label = f"{model.names[cls_id]} {conf:.2f}"

                # Draw rectangle + label
                cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(
                    img,
                    label,
                    (x1, max(y1 - 10, 0)),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.5,
                    (0, 255, 0),
                    2,
                )

        # Save annotated image
        annotated_img_path = os.path.join(output_path, img_file)
        cv2.imwrite(annotated_img_path, cv2.cvtColor(img, cv2.COLOR_RGB2BGR))
        annotated_images.append(img)

# Display images in a grid
if annotated_images:
    cols = 3
    rows = (len(annotated_images) + cols - 1) // cols
    plt.figure(figsize=(15, 5 * rows))

    for i, img in enumerate(annotated_images):
        plt.subplot(rows, cols, i + 1)
        plt.imshow(img)
